In [ ]:
#импорт необходимых библиотек:
#sklearn.neighbors - для реализации метода kNN
#numpy - для работы с данными в векторизованном формате
#пандас - для считывания данных формата csv и построения первых выводов о нашей выборке
import warnings
warnings.filterwarnings('ignore')
from sklearn import neighbors
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('advert.csv') #считываем данные из файла

In [ ]:
df.head() #первые пять строк 

Строго говоря, первые пять строк нам могут лишь показать совсем малую часть информации, которую содержит наша выборка. Конкретно в этом случае, видим, что ключевым признаком является столбец Sales_success. Два других - TV и Newspaper являются признаками, на которых будет обучаться наша модель. Для того, чтобы алгоритм kNN, да и вообще любой алгоритм работал корректно, необходимо провести анализ и соответствующую предобработку данных. Так как признаков в нашей выборке не так много, для анализа достаточно вывести на экран распределение наших признаков по выборке.

In [ ]:
plt.figure()
df.drop(columns = ['Sales_success']).plot(figsize = (20,10))

На приведенном выше графике явно заметно, что признаки, на которых будет обучаться наша модель независимы друг от друга, иначе говоря, отсутствует какая-либо корреляция между ними, но вот масштаб явно разный, подтвердим это конкретными цифрами:

##### Максимальные и минимальные значения признаков

In [ ]:
print('TV max =', df['TV'].max())
print('TV min =', df['TV'].min())
print('Newspaper max =', df['Newspaper'].max())
print('Newspaper min =', df['Newspaper'].min())
print('TV mean =', df['TV'].mean())
print('Newspaper mean =', df['Newspaper'].mean())

И действительно, разница между максимальными и средними значениями велика. Отсутствие разницы между минимальными значениями при наличии вышеупомянутых аргументов - не критично, так как уже в любом случае необходимо произвести предобработку наших данных.
Кроме того, для того, чтобы определить оптимальное количество ближайших соседей, мы воспользуемся таким способом подбора k, как grid search, а также кросс-валидацией, для оптимизации тренировочного процесса, ну и  разделим наш датасет на тренировочную и тестовую выборки, для того, чтобы в последствии оценить работу алгоритма.

In [ ]:
#создаем тренировочную и тестовую выборки в нужном нам форматах
X = np.array(df.drop(columns = ['Sales_success']))
Y = np.array(df['Sales_success']).reshape((200,1))
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X,Y)
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled,Y, test_size = 0.2)
#print(X[:,0].min(),X[:,0].max())
#print(X[:,1].min(),X[:,1].max())
print('X_train shape =',X_train.shape)
print('X_test shape =',X_test.shape)
print('Y_train shape =', Y_train.shape)
print('Y_test shape =',Y_test.shape)
print('Вывод уникальных значений ключевого признака :',np.unique(Y))
print('Баланс классов :',list(Y).count(0),list(Y).count(1))

In [ ]:
#создаем model instance
kNN = neighbors.KNeighborsClassifier()
#количество параметров в нашей сетке (количество проверяемых значений k)
param_grid = {'n_neighbors': np.arange(1, 25)}
#тренируем модель на вышеобозначенной сетке
kNN_gscv = GridSearchCV(kNN, param_grid, cv=5)
kNN_gscv.fit(X_train, Y_train)

In [ ]:
print(kNN_gscv.best_score_)
print(kNN_gscv.best_params_)

In [ ]:
kNN_gscv.score(X_test, Y_test)

In [ ]:
def get_graph_params(grid_scores_list):
    params_list = np.zeros((len(grid_scores_list),2))
    for i in range(len(grid_scores_list)):
        params_list[i][0] = grid_scores_list[i][1]
        params_list[i][1] = (i + 1)
    return params_list

In [ ]:
param_list = get_graph_params(kNN_gscv.grid_scores_)

In [ ]:
#выводим на экран график accuracy в зависимости от числа соседей
%matplotlib inline
fig = plt.figure(figsize=(12,8))
plt.xlabel('Значения параметра k')
plt.ylabel('Значение accuracy, соответствующее значению k')
plt.plot(param_list[:,1],param_list[:,0], markevery = [list(param_list[:,0]).index(param_list[:,0].max())], marker="o")
plt.show()

In [ ]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets

n_neighbors = kNN_gscv.best_params_.get('n_neighbors')

h = .02  # step size in the mesh

# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])


# we create an instance of Neighbours Classifier and fit the data.
clf = neighbors.KNeighborsClassifier(n_neighbors = n_neighbors)
clf.fit(X_train, Y_train)

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
shaped = Y.reshape(200,)
# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize = (12,12))
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

# Plot also the training points
plt.scatter(X[:, 0], X[:, 1], c = shaped, cmap=cmap_bold,
                edgecolor='k', s=20)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("2-Class classification (k = %i, weights = '%s')"
              % (19, weights))

plt.show()